In [ ]:
import cv2
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource

def cvt2bokeh(img):
    h, w = img.shape[:2]
    container = np.empty((h, w), dtype=np.uint32)
    view = container.view(dtype=np.uint8).reshape((h, w, 4))
    if img.ndim == 2: 
        view[:, :, 0] = img[::-1, :]  
        view[:, :, 1] = img[::-1, :]  
        view[:, :, 2] = img[::-1, :]  
    else: 
        view[:, :, 0] = img[::-1, :, 0] 
        view[:, :, 1] = img[::-1, :, 1] 
        view[:, :, 2] = img[::-1, :, 2] 
    view[:, :, 3] = 255
    return container

def fundus_roi(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_image)
    threshold = 15
    _, roi = cv2.threshold(v, threshold, 1, cv2.THRESH_BINARY)
    roi = roi.astype(bool)
    white_pixels = np.argwhere(roi == 1)
    if white_pixels.size == 0:
        print("Aucun pixel blanc trouvé dans le masque.")
        return {"roi": roi, "diameter": 0, "image": image}
    x_min, y_min = np.min(white_pixels, axis=0)
    x_max, y_max = np.max(white_pixels, axis=0)
    diameter_x = x_max - x_min
    diameter_y = y_max - y_min
    diameter = int(np.maximum(diameter_x, diameter_y))
    return {"roi": roi, "diameter": diameter, "image": image}

def load_ben_color(image, roi_mask=None):
    sigmaX = 10
    image_roi = image.copy()
    image_roi[~roi_mask] = 0 
    image_roi = cv2.addWeighted(image_roi, 4, cv2.GaussianBlur(image_roi, (0, 0), sigmaX), -4, 128)
    return {"image": image_roi}

def histogram_equalization(img, roi_mask=None):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_roi = gray.copy()
    gray_roi[~roi_mask] = 0  

    cdf = np.cumsum(np.histogram(gray_roi, 1024, [0, 1024])[0])
    cdf = cdf / cdf[-1]
    
    equalized_img = np.interp(gray, np.arange(0, 1024), cdf)
    equalized_img[~roi_mask] = 0 
    equalized_img = np.array(equalized_img * 255, dtype='uint8')
    
    return {"image": equalized_img}

def histogram_equalization_METH2(img, roi_mask=None):
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img_bgr_roi = img_bgr.copy()
    img_bgr_roi[~roi_mask] = 0 

    ycrcb_img = cv2.cvtColor(img_bgr_roi, cv2.COLOR_BGR2YCrCb)
    ycrcb_img[:, :, 0] = cv2.equalizeHist(ycrcb_img[:, :, 0])
    
    equalized_img_bgr = cv2.cvtColor(ycrcb_img, cv2.COLOR_YCrCb2BGR)
    equalized_img_rgb = cv2.cvtColor(equalized_img_bgr, cv2.COLOR_BGR2RGB)
    
    return {"image": equalized_img_rgb}

image_path = "../data/image_6.jpg"
original_image = cv2.imread(image_path)
if original_image is None:
    print(f"Erreur: Impossible de charger l'image depuis {image_path}")
else:
    roi_info = fundus_roi(original_image)
    roi_mask = roi_info['roi']

    processed_image_1 = load_ben_color(original_image, roi_mask)
    equalized_img_1 = histogram_equalization(processed_image_1['image'], roi_mask)
    equalized_img_2 = histogram_equalization_METH2(processed_image_1['image'], roi_mask)

    
    image_original_bokeh = cvt2bokeh(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
    image_processed_bokeh = cvt2bokeh(cv2.cvtColor(processed_image_1['image'], cv2.COLOR_BGR2RGB))
    image_equalized_1_bokeh = cvt2bokeh(equalized_img_1['image'])
    image_equalized_2_bokeh = cvt2bokeh(cv2.cvtColor(equalized_img_2['image'], cv2.COLOR_RGB2BGR))

    output_notebook()

    source1 = ColumnDataSource(data=dict(image=[image_original_bokeh]))
    source2 = ColumnDataSource(data=dict(image=[image_processed_bokeh]))
    source3 = ColumnDataSource(data=dict(image=[image_equalized_1_bokeh]))
    source4 = ColumnDataSource(data=dict(image=[image_equalized_2_bokeh]))

    image_height, image_width = original_image.shape[:2]

    p1 = figure(title="Image originale", 
                x_range=(0, image_width), y_range=(0, image_height),
                width=image_width//10, height=image_height//10)
    p2 = figure(title="Image traitée (load_ben_color)", 
                x_range=p1.x_range, y_range=p1.y_range,
                width=image_width//10, height=image_height//10)
    p3 = figure(title="Image égalisée avec méthode 1", 
                x_range=p1.x_range, y_range=p1.y_range,
                width=image_width//10, height=image_height//10)
    p4 = figure(title="Image égalisée avec méthode 2", 
                x_range=p1.x_range, y_range=p1.y_range,
                width=image_width//10, height=image_height//10)

    p1.image_rgba(image='image', source=source1, x=0, y=0, dw=image_width, dh=image_height)
    p2.image_rgba(image='image', source=source2, x=0, y=0, dw=image_width, dh=image_height)
    p3.image_rgba(image='image', source=source3, x=0, y=0, dw=image_width, dh=image_height)
    p4.image_rgba(image='image', source=source4, x=0, y=0, dw=image_width, dh=image_height)

    grid = gridplot([[p1, p2], [p3, p4]])
    show(grid)

